This script processes Alex's Berkeley vision dataset into the correct format .hf5
Saved at '/Users/jiamingxu/Desktop/Language_straightenting/data/vision_fixation'

In [89]:
import os
os.chdir('/Users/jiamingxu/Desktop/Language_straightenting/data/bootcamp-stuff/bootcamp')

In [90]:
import h5py
import cortex
import numpy as np      
import itertools
from utils import draw_slices

In [91]:
# load data
template = np.load("template.npy")
resp_raw = h5py.File('validation_notavg.hf5','r+')['data'][:, :31]
# re-order data
valorder = [2, 1, 3, 2, 3, 1, 3, 2, 1, 3, 1, 2, 3, 1, 2, 1, 2, 3, 2, 1, 3, 3, 2, 1, 1, 2, 3, 1, 2, 3]
numblocks = len(valorder)
blocklen = 30

In [92]:
# get mask
subject, xfm = 'S1', 'fullhead'
mask = cortex.db.get_mask(subject, xfm)
mask.shape, np.sum(mask)

((31, 100, 100), 80615)

In [81]:
nan_idx = np.where(np.all(np.isnan(resp_raw), axis = 0))
nan_mask = np.ones_like(mask, dtype = bool) 
nan_mask[nan_idx] = False

In [93]:
nRep, TR, Z, Y, X, = 10, 90, 31, 100, 100
resp_vol = np.zeros((nRep, TR, Z, Y, X))

for sl in range(Z):
    slicedata = resp_raw[:,sl,:,:]
    resp_sl = np.zeros((nRep,TR,Y,X))
    for y,x in ((y,x) for x in range(X) for y in range(Y)):
        data = slicedata[:,y,x] 

        ## Reshape and reorder the data
        rsdata = data.reshape(numblocks, blocklen) #30 blocks * 30 TRs
        blocksets = [[], [], []]
        for blocknum, blockset in enumerate(valorder):
            blocksets[blockset-1].append(rsdata[blocknum])
        
        ## Combine the reordered data for each pixel
        rodata = np.hstack([np.vstack(b) for b in blocksets]) #(10,90)
        resp_sl[:,:,y,x] = rodata
        
    ## Store reordered responses in the big volume
    resp_vol[:, :, sl, :, :] = resp_sl

In [94]:
# apply mask
# combined_mask = mask & nan_mask
# resp = resp_vol[:,:,combined_mask]
resp_vol_nonan = np.nan_to_num(resp_vol, nan=0)
resp = resp_vol_nonan[:,:,mask]

In [86]:
np.sum(mask == True)

80615

In [95]:
# save data
output_dir = '/Users/jiamingxu/Desktop/Language_straightenting/data/vision_fixation'
output_file = 'vision_fixation_resp.hf5'
mask_file = 'nan_mask.npy'
output_path = os.path.join(output_dir, output_file)
mask_path = os.path.join(output_dir, mask_file)
os.makedirs(output_dir, exist_ok = True)
with h5py.File(output_path,'w') as f:
    f.create_dataset('data', data=resp)
    
#np.save(mask_path, nan_mask)